# 18 激活函数与GPU加速

## activation function

* <font face="黑体" color=green size=5>sigmoid</font>
* <font face="黑体" color=green size=5>tanh</font>
<img src='./images/15.png' style="zoom:50%">
* <font face="黑体" color=green size=5>ReLU</font>
<img src='./images/16.png' style="zoom:50%">
* <font face="黑体" color=green size=5>Leaky ReLU</font>
<img src='./images/17.png' style="zoom:50%">
* <font face="黑体" color=green size=5>SELU</font>
<img src='./images/18.png' style="zoom:50%">
* <font face="黑体" color=green size=5>softplus</font>
<img src='./images/19.png' style="zoom:50%">

## GPU accelerate

```python
device = torch.device('cuda:0')
```

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [2]:
batch_size=200
learning_rate=0.01
epochs=10

In [3]:
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('./data', train=True, download=True,
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('./data', train=False, transform=transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.1307,), (0.3081,))
    ])),
    batch_size=batch_size, shuffle=True)

### 使用LeakyReLU激活函数

In [4]:
class MLP(nn.Module):

    def __init__(self):
        super(MLP, self).__init__()

        self.model = nn.Sequential(
            nn.Linear(784, 200),
            nn.LeakyReLU(inplace=True),
            nn.Linear(200, 200),
            nn.LeakyReLU(inplace=True),
            nn.Linear(200, 10),
            nn.LeakyReLU(inplace=True),
        )

    def forward(self, x):
        x = self.model(x)

        return x

### gpu accelerate: ctrl f :device,自己查看

In [5]:
device = torch.device('cuda:0')
net = MLP().to(device)
optimizer = optim.SGD(net.parameters(), lr=learning_rate)
criteon = nn.CrossEntropyLoss().to(device)

In [6]:
for epoch in range(epochs):

    for batch_idx, (data, target) in enumerate(train_loader):
        data = data.view(-1, 28*28)
        data, target = data.to(device),target.cuda()   # 通常使用.to(device),.cuda()老版本，不建议使用。
                                                       # 对于data有可能产生cpu tensor 和 gpu tensor，确保使用的是否一致。
        logits = net(data)
        loss = criteon(logits, target)

        optimizer.zero_grad()
        loss.backward()
        # print(w1.grad.norm(), w2.grad.norm())
        optimizer.step()

        if batch_idx % 100 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                       100. * batch_idx / len(train_loader), loss.item()))


    test_loss = 0
    correct = 0
    for data, target in test_loader:
        data = data.view(-1, 28 * 28)
        data, target = data.to(device), target.cuda()
        
        logits = net(data)
        test_loss += criteon(logits, target).item()

        pred = logits.data.max(1)[1]
        correct += pred.eq(target.data).sum()

    test_loss /= len(test_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

Train Epoch: 0 [0/60000 (0%)]	Loss: 2.303960
Train Epoch: 0 [20000/60000 (33%)]	Loss: 1.984523
Train Epoch: 0 [40000/60000 (67%)]	Loss: 1.459735

Test set: Average loss: 0.0063, Accuracy: 6290/10000 (62%)

Train Epoch: 1 [0/60000 (0%)]	Loss: 1.351230
Train Epoch: 1 [20000/60000 (33%)]	Loss: 1.025896
Train Epoch: 1 [40000/60000 (67%)]	Loss: 0.927920

Test set: Average loss: 0.0043, Accuracy: 7323/10000 (73%)

Train Epoch: 2 [0/60000 (0%)]	Loss: 0.748525
Train Epoch: 2 [20000/60000 (33%)]	Loss: 0.796929
Train Epoch: 2 [40000/60000 (67%)]	Loss: 0.563963

Test set: Average loss: 0.0020, Accuracy: 8923/10000 (89%)

Train Epoch: 3 [0/60000 (0%)]	Loss: 0.438475
Train Epoch: 3 [20000/60000 (33%)]	Loss: 0.445699
Train Epoch: 3 [40000/60000 (67%)]	Loss: 0.314467

Test set: Average loss: 0.0017, Accuracy: 9043/10000 (90%)

Train Epoch: 4 [0/60000 (0%)]	Loss: 0.368438
Train Epoch: 4 [20000/60000 (33%)]	Loss: 0.437965
Train Epoch: 4 [40000/60000 (67%)]	Loss: 0.326460

Test set: Average loss: 0.0015

### Tips:GPU训练比CPU训练效果要好，需要探明其缘由。。。

### caculate accuracy

In [7]:
logits = torch.rand(4, 10)

In [8]:
pred = F.softmax(logits, dim=1)
pred.shape

torch.Size([4, 10])

In [9]:
pred_label = pred.argmax(dim=1)
pred_label

tensor([4, 4, 2, 5])

In [10]:
logits.argmax(dim=1)

tensor([4, 4, 2, 5])

In [11]:
label = torch.tensor([4,8,3,8])
correct = torch.eq(pred_label, label)
correct

tensor([1, 0, 0, 0], dtype=torch.uint8)

In [12]:
correct.sum().float().item()/4.

0.25